In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from __future__ import print_function,division

In [2]:
df = pd.read_csv("/Users/chaoxu/code/local-spark/Data/qiancheng_sample_new.csv")

IOError: File /Users/chaoxu/code/local-spark/Data/qiancheng_sample_new.csv does not exist

In [ ]:
df_ios_test = pd.read_csv("/Users/chaoxu/code/local-spark/Data/qiancheng_ios_vld.csv")

In [ ]:
# df_ios_test.dtypes

In [ ]:
df_android_test = pd.read_csv("/Users/chaoxu/code/local-spark/Data/qiancheng_android_vld.csv")

In [ ]:
# df_android_test.timestamp.dtypes

In [ ]:
df.os.value_counts()

In [ ]:
def resample(data, positiveRatio=0.04):
    positive = data[data["label"] == 1]
    negative = data[data["label"] == 0]
    n = int((1 - positiveRatio) / positiveRatio) * positive.shape[0]
    print(n)
    _negative = negative.sample(n=n)
    df = pd.concat([positive, _negative])
    return df

In [ ]:
df[['label']]=df[['label']].fillna(0)

In [ ]:
df_android_test[['label']]=df_android_test[['label']].fillna(0)

In [ ]:
df_ios_test[['label']]=df_ios_test[['label']].fillna(0)

In [ ]:
from datetime import datetime, timedelta
df["timestamp"] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
df["timestamp"] = pd.DatetimeIndex(df["timestamp"]) + timedelta(hours=8)
df = df.sort_values(by="timestamp")
df = df.drop(['idcIP.anomaly', 'cracked.anomaly', 'uaMismatch.anomaly',"timestamp","maxent_id","timestamp",'ipSeg24','ipGeo','event_id','scenario'],axis=1)
df[['label']]=df[['label']].fillna(0)
df[['label']] = df[['label']].astype(int)
df = df.loc[df['event_type'] =='ACT']
df = df.drop(['event_type'],axis=1)

In [ ]:
df_android_test["timestamp"] = pd.to_datetime(df_android_test['timestamp'], unit='ms', utc=True)
df_android_test["timestamp"] = pd.DatetimeIndex(df_android_test["timestamp"]) + timedelta(hours=8)
df_android_test = df_android_test.sort_values(by="timestamp")
df_android_test = df_android_test.drop(['idcIP.anomaly', 'cracked.anomaly', 'uaMismatch.anomaly',"timestamp","maxent_id","timestamp",'ipSeg24','ipGeo','event_id','scenario'],axis=1)
df_android_test[['label']]=df_android_test[['label']].fillna(0)
df_android_test[['label']] = df_android_test[['label']].astype(int)
df_android_test = df_android_test.loc[df_android_test['event_type'] =='ACT']
df_android_test = df_android_test.drop(['event_type'],axis=1)

In [ ]:
df_ios_test["timestamp"] = pd.to_datetime(df_ios_test['timestamp'], unit='ms', utc=True)
df_ios_test["timestamp"] = pd.DatetimeIndex(df_ios_test["timestamp"]) + timedelta(hours=8)
df_ios_test = df_ios_test.sort_values(by="timestamp")
df_ios_test = df_ios_test.drop(['idcIP.anomaly', 'cracked.anomaly', 'uaMismatch.anomaly',"timestamp","maxent_id","timestamp",'ipSeg24','ipGeo','event_id','scenario'],axis=1)
df_ios_test[['label']]=df_ios_test[['label']].fillna(0)
df_ios_test[['label']] = df_ios_test[['label']].astype(int)
df_ios_test = df_ios_test.loc[df_ios_test['event_type'] =='ACT']
df_ios_test = df_ios_test.drop(['event_type'],axis=1)

In [ ]:
import re
anormaly = re.compile('.*anomaly$')
anormaly_match = np.vectorize(lambda x:bool(anormaly.match(x)))
anormaly_cols = df.columns.values[anormaly_match(df.columns.values)]

value = re.compile('.*value$')
value_match = np.vectorize(lambda x:bool(value.match(x)))
value_cols = df.columns.values[value_match(df.columns.values)]

count = re.compile('.*counts$')
count_match = np.vectorize(lambda x:bool(count.match(x)))
count_cols = df.columns.values[count_match(df.columns.values)]

loan = re.compile('.*loan$')
loan_match = np.vectorize(lambda x:bool(loan.match(x)))
loan_cols = df.columns.values[loan_match(df.columns.values)]

In [ ]:
cols = df.columns
fig, ax = plt.subplots(figsize=(16,9))
ax.barh(range(len(cols)),df[cols].isnull().sum()/df.shape[0])
ax.set_yticks(range(len(cols)))
ax.set_yticklabels(cols, size=14)
ax.set_title('features nan percentage',size= 18)
plt.show()

In [ ]:
df[loan_cols]=df[loan_cols].fillna(0)
df_android_test[loan_cols]=df_android_test[loan_cols].fillna(0)
df_ios_test[loan_cols]=df_ios_test[loan_cols].fillna(0)

In [ ]:
df[anormaly_cols] = df[anormaly_cols].fillna(1)
df[value_cols] = df[value_cols].fillna(1)
df_android_test[anormaly_cols] = df_android_test[anormaly_cols].fillna(1)
df_android_test[value_cols] = df_android_test[value_cols].fillna(1)
df_ios_test[anormaly_cols] = df_ios_test[anormaly_cols].fillna(1)
df_ios_test[value_cols] = df_ios_test[value_cols].fillna(1)

In [ ]:
obj_df = df.select_dtypes(include=['object'])
obj_df.columns

In [ ]:
bool_df = df.select_dtypes(include=['bool'])
bool_df.columns

In [ ]:
df[count_cols]=df[count_cols].fillna(0)
df_android_test[count_cols]=df_android_test[count_cols].fillna(0)
df_ios_test[count_cols]=df_ios_test[count_cols].fillna(0)

In [ ]:
def show_missing():
    missing = df.columns[df.isnull().any()].tolist()
    return missing
df[show_missing()].isnull().sum()

整体的label平衡性

In [ ]:
df.label.value_counts(normalize=True)

In [ ]:
df.label.value_counts(normalize=True).plot(kind='bar')
plt.show()

In [ ]:
df_ios = df.loc[df.os == 'ios']

In [ ]:
# df_ios = resample(df_ios)

In [ ]:
df_ios.head(10)

In [ ]:
df_ios.dtypes

In [ ]:
obj_cols = obj_df.columns.values.tolist()
print(obj_cols)
bool_cols = bool_df.columns.values.tolist()
print(bool_cols)
encoder_cols = obj_cols + bool_cols
print(encoder_cols)

In [ ]:
from sklearn.preprocessing import LabelEncoder
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:
df_ios = MultiColumnLabelEncoder(columns = encoder_cols).fit_transform(df_ios)
df_ios_test = MultiColumnLabelEncoder(columns = encoder_cols).fit_transform(df_ios_test)

In [ ]:
df_ios.head()

In [ ]:
df_ios.label.value_counts(normalize=True).plot(kind='bar')
plt.show()

In [ ]:
def show_missing():
    missing = df.columns[df.isnull().any()].tolist()
    return missing
df_ios[show_missing()].isnull().sum()

In [ ]:
# cols = df_ios.columns
# fig, ax = plt.subplots(figsize=(16,9))
# ax.barh(range(len(cols)),df_ios[cols].isnull().sum()/df_ios.shape[0])
# ax.set_yticks(range(len(cols)))
# ax.set_yticklabels(cols, size=14)
# ax.set_title('features nan percentage',size= 18)
# plt.show()

In [ ]:
ios_drop = ['os','aid_loan','imei_loan','mac_loan','imei_counts','mac_counts','aid_counts']
df_ios.drop(ios_drop,axis=1)
df_ios_test.drop(ios_drop,axis=1)

In [ ]:
df_ios.label.value_counts(normalize=True).plot(kind='bar')
plt.show()

In [ ]:
# X_ios = df_ios.ix[:, df_ios.columns != 'label']
# y_ios = df_ios.ix[:, df_ios.columns == 'label']

In [ ]:
# from sklearn.cross_validation import train_test_split
# def splitData(df, ratio):
#     """
#     """
#     positiveData = df[df["label"] == 1].copy(deep=True)
#     _positiveTrain, _positiveTest = train_test_split(positiveData, train_size=ratio)
#     negativeData = df[df["label"] == 0].copy(deep=True)
#     _negativeTrain, _negativeTest = train_test_split(negativeData, train_size=ratio)
#     train_set = pd.concat([_positiveTrain, _negativeTrain])
#     test_set = pd.concat([_positiveTest, _negativeTest])
#     return train_set, test_set

df_ios_train = df_ios
X_ios_train = df_ios_train.ix[:, df_ios_train.columns != 'label']
X_ios_test  = df_ios_test.ix[:, df_ios_test.columns != 'label']
y_ios_train = df_ios_train.ix[:, df_ios_train.columns == 'label']
y_ios_test  = df_ios_test.ix[:, df_ios_test.columns == 'label']
print("Number transactions ios train dataset: ", X_ios_train.shape[0])
print("Number transactions ios train dataset: ", y_ios_train.shape[0])
print("Number transactions ios test dataset: ", X_ios_test.shape[0])
print("Total number of ios transactions: ", X_ios_train.size)

In [ ]:
X_ios_test.columns

In [ ]:
from __future__ import print_function,division
import itertools
import matplotlib.pylab as plt
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    else:
        1#print('Confusion matrix, without normalization')

    #print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('label 1')
    plt.xlabel('label 0')

In [ ]:
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,precision_score,classification_report
from sklearn.ensemble import RandomForestClassifier
params = {'max_features':'auto', 'max_depth': 3, 'min_samples_split': 5, \
            'class_weight':'balanced','oob_score':True}
params['n_estimators'] = 2
params['n_jobs'] = 50
forest = RandomForestClassifier(**params)
clf_a = forest.fit(X_ios_train,y_ios_train.values.ravel())
y_ios_pred = clf_a.predict(X_ios_test)

print(X_ios_test.size)
print(y_ios_test.size)
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_ios_test,y_ios_pred)
np.set_printoptions(precision=2)
np.set_printoptions(precision=2)
recall = cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1])
precision = cnf_matrix[1,1]/(cnf_matrix[0,1]+cnf_matrix[1,1])
f2 = 0.2*(4/recall + 1/precision)
print("Recall metric in the testing dataset: ", recall)
print("Precision metric in the testting dataset:", precision)
print("f2 metric in the testting dataset:", f2)
# Plot non-normalized confusion matrix
class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix
                      , classes=class_names
                      , title='Confusion matrix')
plt.show()

print(classification_report(y_ios_test, y_ios_pred, target_names=['0','1']))

In [ ]:
sum_ios_neg = y_ios_train.loc[y_ios_train.label == 1].shape[0]
sum_ios_pos = y_ios_train.loc[y_ios_train.label == 0].shape[0]
scale_ios_ratio = sum_ios_neg / sum_ios_pos

In [ ]:
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb

In [ ]:
xgb_ios_params1 = {
    'objective' : 'binary:logistic',
#     'objective' : 'binary:logitraw',
    'nthread':4,
#     'num_class': 2,
    'scale_pos_weight':scale_ios_ratio,
#     'missing':-6.666,
    'seed':27 
}

In [ ]:
cv_params = {
    'pre_dispatch': '2*n_jobs', 
    'refit':True,
    'scoring':'roc_auc',
#     'scoring':'f1_macro',
#     'scoring':'average_precision',
#     'scoring':'accuracy',
#     'scoring':'recall',
    'cv':5,
    'verbose':0
}

In [ ]:
param_ios_test2 = {
    'n_estimators':range(10,200,10)
}
params_ios = {}
params_ios.update(xgb_ios_params1)
xgb_params2 = params_ios
xgb_params2

In [ ]:
gsearch1_ios = GridSearchCV(estimator = xgb.XGBClassifier(**xgb_params2),
                        param_grid=param_ios_test2,**cv_params)

In [ ]:
X_ios_cols = X_ios_train.columns
gsearch1_ios.fit(X_ios_train[X_ios_cols],y_ios_train['label'])

In [ ]:
gsearch1_ios.grid_scores_, gsearch1_ios.best_params_, gsearch1_ios.best_score_

In [ ]:
params_ios.update(gsearch1_ios.best_params_)
params_ios

In [ ]:
xgb_ios_params3 = params_ios
param_ios_test3 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}

In [ ]:
gsearch3_ios = GridSearchCV(estimator = xgb.XGBClassifier(**xgb_ios_params3),
                        param_grid=param_ios_test3,**cv_params)

In [ ]:
gsearch3_ios.fit(X_ios_train[X_ios_cols],y_ios_train['label'])

In [ ]:
gsearch3_ios.grid_scores_, gsearch3_ios.best_params_, gsearch3_ios.best_score_

In [ ]:
params_ios.update(gsearch3_ios.best_params_)
params_ios

In [ ]:
xgb_ios_params4 = params_ios
param_ios_test4 = {
    'gamma':[i/10.0 for i in range(0,5)]
}

In [ ]:
gsearch4_ios = GridSearchCV(estimator = xgb.XGBClassifier(**xgb_ios_params4),
                        param_grid=param_ios_test4,**cv_params)

In [ ]:
gsearch4_ios.fit(X_ios_train[X_ios_cols],y_ios_train['label'])

In [ ]:
gsearch4_ios.grid_scores_, gsearch4_ios.best_params_, gsearch4_ios.best_score_

In [ ]:
params_ios.update(gsearch4_ios.best_params_)
params_ios

In [ ]:
xgb_ios_params5 = params_ios
param_ios_test5 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)],
}

In [ ]:
gsearch5_ios = GridSearchCV(estimator = xgb.XGBClassifier(**xgb_ios_params5),
                        param_grid=param_ios_test5,**cv_params)

In [ ]:
gsearch5_ios.fit(X_ios_train[X_ios_cols],y_ios_train['label'])

In [ ]:
gsearch5_ios.grid_scores_, gsearch5_ios.best_params_, gsearch5_ios.best_score_

In [ ]:
params_ios.update(gsearch5_ios.best_params_)
params_ios

In [ ]:
xgb_ios_params6 = params_ios
param_ios_test6 = {
    'reg_lambda':[0, 0.001, 0.005, 0.01, 0.05],
}
gsearch6_ios = GridSearchCV(estimator = xgb.XGBClassifier(**xgb_ios_params6),
                        param_grid=param_ios_test6,**cv_params)
gsearch6_ios.fit(X_ios_train[X_ios_cols],y_ios_train['label'])
gsearch6_ios.grid_scores_, gsearch6_ios.best_params_, gsearch6_ios.best_score_
params_ios.update(gsearch6_ios.best_params_)
params_ios

In [ ]:
l_np = np.arange(0.1,0.2,0.02).tolist()
xgb_ios_params7 = params_ios
param_ios_test7 = {
    'learning_rate':l_np
}
gsearch7_ios = GridSearchCV(estimator = xgb.XGBClassifier(**xgb_ios_params7),
                        param_grid=param_ios_test7,**cv_params)
gsearch7_ios.fit(X_ios_train[X_ios_cols],y_ios_train['label'])
gsearch7_ios.grid_scores_, gsearch7_ios.best_params_, gsearch7_ios.best_score_
params_ios.update(gsearch7_ios.best_params_)
params_ios

In [ ]:
scores = np.arange(0.1,1.0,0.1).tolist()
xgb_ios_params8 = params_ios
param_ios_test8 = {
    'base_score':scores
}
gsearch8_ios = GridSearchCV(estimator = xgb.XGBClassifier(**xgb_ios_params8),
                        param_grid=param_ios_test8,**cv_params)
gsearch8_ios.fit(X_ios_train[X_ios_cols],y_ios_train['label'])
gsearch8_ios.grid_scores_, gsearch8_ios.best_params_, gsearch8_ios.best_score_
params_ios.update(gsearch8_ios.best_params_)
params_ios

In [ ]:
from mlxtend.plotting import plot_learning_curves
xgb_ios_final = xgb.XGBClassifier(**params_ios)
y_ios_train_f = y_ios_train.label.astype(float)
y_ios_test_f = y_ios_test.label.astype(float)
plot_learning_curves(X_ios_train, y_ios_train_f, X_ios_test, y_ios_test_f, xgb_ios_final, print_model=False, style='ggplot')
plt.show()

In [ ]:
fig,ax= plt.subplots(figsize=(10,10))
xgb.plot_importance(xgb_ios_final,ax=ax)
plt.show()

In [ ]:
fig,ax= plt.subplots(figsize=(10,10))
# xgb.plot_tree(xgb_ios_final,num_trees=59,ax=ax)
xgb.plot_tree(xgb_ios_final,ax=ax)
plt.show()

In [ ]:
xgb_ios_final.fit(X_ios_train, y_ios_train)
y_ios_pred = xgb_ios_final.predict(X_ios_test)
cnf_matrix = confusion_matrix(y_ios_test,y_ios_pred)
np.set_printoptions(precision=2)
np.set_printoptions(precision=2)
recall = cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1])
precision = cnf_matrix[1,1]/(cnf_matrix[0,1]+cnf_matrix[1,1])
f2 = 0.2*(4/recall + 1/precision)
print("Recall metric in the testing dataset: ", recall)
print("Precision metric in the testting dataset:", precision)
print("f2 metric in the testting dataset:", f2)
# Plot non-normalized confusion matrix
class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix
                      , classes=class_names
                      , title='Confusion matrix')
plt.show()

print(classification_report(y_ios_test, y_ios_pred, target_names=['0','1']))

In [ ]:
df_android = df.loc[df.os == 'android']
df_android = MultiColumnLabelEncoder(columns = encoder_cols).fit_transform(df_android)
df_android_test = MultiColumnLabelEncoder(columns = encoder_cols).fit_transform(df_android_test)

In [ ]:
df_android.head()

In [ ]:
df_android.label.value_counts(normalize=True).plot(kind='bar')
plt.show()

In [ ]:
df_android[show_missing()].isnull().sum()

In [ ]:
df_android['proxyIP.value'].unique()

In [ ]:
android_drop = ['os','idfa_loan','idfa_counts', u'idfv_counts','imei_loan']
df_android =df_android.drop(android_drop,axis=1)
df_android_test =df_android_test.drop(android_drop,axis=1)

In [ ]:
df_android_train = df_android
X_android_train = df_android_train.ix[:, df_android_train.columns != 'label']
X_android_test  = df_android_test.ix[:,  df_android_test.columns != 'label']
y_android_train = df_android_train.ix[:, df_android_train.columns == 'label']
y_android_test  = df_android_test.ix[:,  df_android_test.columns == 'label']
print("Number transactions android train dataset: ", X_android_train.shape[0])
print("Number transactions android train dataset: ", y_android_train.shape[0])
print("Number transactions android test dataset: ", X_android_test.shape[0])
print("Total number of android transactions: ", X_android_train.size)

In [ ]:
y_android_test.label.value_counts()

In [ ]:
sum_android_neg = y_android_train.loc[y_android_train.label == 1].shape[0]
sum_android_pos = y_android_train.loc[y_android_train.label == 0].shape[0]
scale_android_ratio = sum_android_neg / sum_android_pos

In [ ]:
xgb_android_params = {
    'objective' : 'binary:logistic',
#     'objective' : 'binary:logitraw',
    'nthread':4,
#     'num_class': 2,
#     'scale_pos_weight':scale_android_ratio,
#     'missing':-6.666,
    'seed':27 
}

In [ ]:
param_android_test1 = {
    'n_estimators':range(10,200,10)
}
params_android = {}
params_android.update(xgb_android_params)
xgb_android_params1 = params_android
xgb_android_params1

In [ ]:
gsearch1_android = GridSearchCV(estimator = xgb.XGBClassifier(**xgb_android_params1),
                        param_grid=param_android_test1,**cv_params)
X_android_cols = X_android_train.columns
gsearch1_android.fit(X_android_train[X_android_cols],y_android_train['label'])
gsearch1_android.grid_scores_, gsearch1_android.best_params_, gsearch1_android.best_score_
params_android.update(gsearch1_android.best_params_)
params_android

In [ ]:
xgb_android_params2 = params_android
param_android_test2 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}
gsearch2_android = GridSearchCV(estimator = xgb.XGBClassifier(**xgb_android_params2),
                        param_grid=param_android_test2,**cv_params)
gsearch2_android.fit(X_android_train[X_android_cols],y_android_train['label'])
gsearch2_android.grid_scores_, gsearch2_android.best_params_, gsearch2_android.best_score_
params_android.update(gsearch2_android.best_params_)
params_android

In [ ]:
xgb_android_params3 = params_android
param_android_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3_android = GridSearchCV(estimator = xgb.XGBClassifier(**xgb_android_params3),
                        param_grid=param_android_test3,**cv_params)
gsearch3_android.fit(X_android_train[X_android_cols],y_android_train['label'])
gsearch3_android.grid_scores_, gsearch3_android.best_params_, gsearch3_android.best_score_
params_android.update(gsearch3_android.best_params_)
params_android

In [ ]:
xgb_android_params4 = params_android
param_android_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)],
}
gsearch4_android = GridSearchCV(estimator = xgb.XGBClassifier(**xgb_android_params4),
                        param_grid=param_android_test4,**cv_params)
gsearch4_android.fit(X_android_train[X_android_cols],y_android_train['label'])
gsearch4_android.grid_scores_, gsearch4_android.best_params_, gsearch4_android.best_score_
params_android.update(gsearch4_android.best_params_)
params_android

In [ ]:
xgb_android_params5 = params_android
param_android_test5 = {
    'reg_lambda':[0, 0.001, 0.005, 0.01, 0.05],
}
gsearch5_android = GridSearchCV(estimator = xgb.XGBClassifier(**xgb_android_params5),
                        param_grid=param_android_test5,**cv_params)
gsearch5_android.fit(X_android_train[X_android_cols],y_android_train['label'])
gsearch5_android.grid_scores_, gsearch5_android.best_params_, gsearch5_android.best_score_
params_android.update(gsearch5_android.best_params_)
params_android

In [ ]:
l_np = np.arange(0.1,0.2,0.02).tolist()
xgb_android_params6 = params_android
param_android_test6 = {
    'reg_lambda':l_np
}
gsearch6_android = GridSearchCV(estimator = xgb.XGBClassifier(**xgb_android_params6),
                        param_grid=param_android_test6,**cv_params)
gsearch6_android.fit(X_android_train[X_android_cols],y_android_train['label'])
gsearch6_android.grid_scores_, gsearch6_android.best_params_, gsearch6_android.best_score_
params_android.update(gsearch6_android.best_params_)
params_android

In [ ]:
scores = np.arange(0.1,1.0,0.1).tolist()
xgb_android_params7 = params_android
param_android_test7 = {
    'base_score':scores
}
gsearch7_android = GridSearchCV(estimator = xgb.XGBClassifier(**xgb_android_params7),
                        param_grid=param_android_test7,**cv_params)
gsearch7_android.fit(X_android_train[X_android_cols],y_android_train['label'])
gsearch7_android.grid_scores_, gsearch7_android.best_params_, gsearch7_android.best_score_
params_android.update(gsearch7_android.best_params_)
params_android

In [ ]:
xgb_android_final = xgb.XGBClassifier(**params_android)
y_android_train_f = y_android_train.label.astype(float)
y_android_test_f = y_android_test.label.astype(float)
plot_learning_curves(X_android_train, y_android_train_f, X_android_test, y_android_test_f, xgb_android_final, print_model=False, style='ggplot')
plt.show()

In [ ]:
fig,ax= plt.subplots(figsize=(10,10))
xgb.plot_importance(xgb_android_final,ax=ax)
plt.show()

In [ ]:
fig,ax= plt.subplots(figsize=(10,10))
# xgb.plot_tree(xgb_ios_final,num_trees=59,ax=ax)
xgb.plot_tree(xgb_android_final,ax=ax)
plt.show()

In [ ]:
xgb_android_final.fit(X_android_train, y_android_train)
y_android_pred = xgb_android_final.predict(X_android_test)
cnf_matrix = confusion_matrix(y_android_test,y_android_pred)
np.set_printoptions(precision=2)
np.set_printoptions(precision=2)
recall = cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1])
precision = cnf_matrix[1,1]/(cnf_matrix[0,1]+cnf_matrix[1,1])
f2 = 0.2*(4/recall + 1/precision)
print("Recall metric in the testing dataset: ", recall)
print("Precision metric in the testting dataset:", precision)
print("f2 metric in the testting dataset:", f2)
# Plot non-normalized confusion matrix
class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix
                      , classes=class_names
                      , title='Confusion matrix')
plt.show()

print(classification_report(y_android_test, y_android_pred, target_names=['0','1']))